# Loan Approval Exploratory and Bias Analysis
## by Albert Hunduza

### 1) Importing dependencies

In [1]:
# importing the required libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


### 2) Loading Dataset

In [2]:
# reading the data
loan_data = pd.read_csv('loans50k.csv', encoding='latin1')
loan_data.head()

,loanID,amount,term,rate,payment,grade,employment,length,home,income,...,totalBal,totalRevLim,accOpen24,avgBal,bcOpen,bcRatio,totalLim,totalRevBal,totalBcLim,totalIlLim
0,188861,8000.0,36 months,0.14,272.07,C,Warehouseman,3 years,RENT,49966.0,...,48054.0,8100.0,8.0,4369.0,43.0,95.7,60629.0,48054.0,1000.0,52529.0
1,517703,11000.0,36 months,0.10,354.89,B,Vice President,10+ years,MORTGAGE,50000.0,...,265838.0,33400.0,4.0,22153.0,19756.0,37.5,297500.0,11844.0,31600.0,0.0
2,268587,35000.0,36 months,0.15,1220.33,D,Owner/Attorney,10+ years,RENT,360000.0,...,850150.0,83000.0,2.0,121450.0,7686.0,83.2,907300.0,173260.0,83000.0,100000.0
3,579902,20000.0,60 months,0.12,447.83,C,Analyst,2 years,MORTGAGE,60000.0,...,20671.0,14200.0,8.0,1723.0,11061.0,1.2,46964.0,20671.0,11200.0,32764.0
4,617630,12000.0,60 months,0.12,266.88,B,medical technician,10+ years,MORTGAGE,64000.0,...,128034.0,14600.0,3.0,18291.0,5904.0,58.4,134600.0,8475.0,14200.0,0.0


In [3]:
# checking the shape of the data
loan_data.shape

(50000, 32)

### Pre-processing the Dataset

* in this section, we look at the quality of the data and check for missing values

In [4]:
# checking the data types of the variables
loan_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 32 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   loanID       50000 non-null  int64  
 1   amount       49999 non-null  float64
 2   term         49999 non-null  object 
 3   rate         49999 non-null  float64
 4   payment      49999 non-null  float64
 5   grade        49999 non-null  object 
 6   employment   47215 non-null  object 
 7   length       47319 non-null  object 
 8   home         49999 non-null  object 
 9   income       49999 non-null  float64
 10  verified     49999 non-null  object 
 11  status       49999 non-null  object 
 12  reason       49999 non-null  object 
 13  state        49999 non-null  object 
 14  debtIncRat   49999 non-null  float64
 15  delinq2yr    49999 non-null  float64
 16  inq6mth      49999 non-null  float64
 17  openAcc      49999 non-null  float64
 18  pubRec       49999 non-null  float64
 19  revo

In [5]:
# checking the dataset for missing values
loan_data.isnull().sum()

loanID            0
amount            1
term              1
rate              1
payment           1
grade             1
employment     2785
length         2681
home              1
income            1
verified          1
status            1
reason            1
state             1
debtIncRat        1
delinq2yr         1
inq6mth           1
openAcc           1
pubRec            1
revolRatio       19
totalAcc          1
totalPaid         1
totalBal          1
totalRevLim       1
accOpen24         1
avgBal            1
bcOpen          489
bcRatio         521
totalLim          1
totalRevBal       1
totalBcLim        1
totalIlLim        1
dtype: int64

* we will take a closer look at the instances with missing values to see how we can address them

In [6]:
# find the row where Amount column is null
loan_data[loan_data['amount'].isnull()]

,loanID,amount,term,rate,payment,grade,employment,length,home,income,...,totalBal,totalRevLim,accOpen24,avgBal,bcOpen,bcRatio,totalLim,totalRevBal,totalBcLim,totalIlLim
34805,656728,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


* we can see that there's one column with most of the attributes missing. It therefore makes sense to just drop it

In [7]:
# drop the row where amount column is null
loan_data.drop(loan_data[loan_data['amount'].isnull()].index, inplace=True)

In [8]:
# find the row where the employment column is null
loan_data[loan_data['employment'].isnull()]

,loanID,amount,term,rate,payment,grade,employment,length,home,income,...,totalBal,totalRevLim,accOpen24,avgBal,bcOpen,bcRatio,totalLim,totalRevBal,totalBcLim,totalIlLim
41,272229,20800.0,36 months,0.17,736.93,D,NaN,NaN,MORTGAGE,75000.0,...,163717.0,24300.0,3.0,13643.0,610.0,90.8,342467.0,43467.0,6600.0,48617.0
51,290384,15625.0,36 months,0.13,523.39,C,NaN,NaN,MORTGAGE,34000.0,...,110268.0,30600.0,2.0,9189.0,5186.0,75.8,137586.0,28003.0,21400.0,18636.0
105,584627,30000.0,60 months,0.13,677.85,C,NaN,NaN,MORTGAGE,334000.0,...,972277.0,1058000.0,1.0,81023.0,10624.0,34.0,1124474.0,972277.0,16100.0,0.0
148,161338,17550.0,60 months,0.14,412.83,C,NaN,NaN,MORTGAGE,44146.0,...,31423.0,25800.0,1.0,3928.0,955.0,95.3,37562.0,31423.0,20300.0,11762.0
150,556421,5400.0,36 months,0.12,178.15,B,NaN,NaN,RENT,28800.0,...,5934.0,12700.0,4.0,659.0,4711.0,37.2,16039.0,5934.0,7500.0,3339.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49915,400200,4125.0,36 months,0.22,157.52,F,NaN,NaN,RENT,13776.0,...,6459.0,9400.0,2.0,1292.0,941.0,87.3,9400.0,6459.0,7400.0,0.0
49933,459040,18000.0,60 months,0.18,455.72,D,NaN,NaN,MORTGAGE,50000.0,...,136401.0,15700.0,5.0,10492.0,3686.0,66.2,146142.0,15346.0,10900.0,8649.0
49956,624458,6400.0,36 months,0.16,223.72,D,NaN,NaN,MORTGAGE,60000.0,...,39083.0,25000.0,5.0,4885.0,6423.0,51.7,62020.0,39083.0,13300.0,37020.0
49996,426438,26050.0,60 months,0.22,719.33,F,NaN,NaN,MORTGAGE,56677.0,...,257712.0,34300.0,10.0,19824.0,1541.0,88.3,282755.0,34757.0,13200.0,21215.0


In [9]:
# calculate the percentage of missing values in employment column
print('Percentage of missing values in employment column: ', loan_data['employment'].isnull().sum() / loan_data.shape[0] * 100,'%')

Percentage of missing values in employment column:  5.568111362227245 %


In [10]:
# calculate the percentage of missing values in length column
print('Percentage of missing values in length column: ', loan_data['length'].isnull().sum() / loan_data.shape[0] * 100, '%')

Percentage of missing values in length column:  5.360107202144042 %


In [11]:
# calculate the percentage of missing values in  revolRatio column
print('Percentage of missing values in revolRatio column: ', loan_data['revolRatio'].isnull().sum() / loan_data.shape[0] * 100, '%')

Percentage of missing values in revolRatio column:  0.03600072001440029 %


* since just 0.036% of values in the revolRatio are missing we can just drop it

In [12]:
# dropp the rows where revolRatio column is null
loan_data.drop(loan_data[loan_data['revolRatio'].isnull()].index, inplace=True)

In [13]:
loan_data.isnull().sum()

loanID            0
amount            0
term              0
rate              0
payment           0
grade             0
employment     2781
length         2679
home              0
income            0
verified          0
status            0
reason            0
state             0
debtIncRat        0
delinq2yr         0
inq6mth           0
openAcc           0
pubRec            0
revolRatio        0
totalAcc          0
totalPaid         0
totalBal          0
totalRevLim       0
accOpen24         0
avgBal            0
bcOpen          472
bcRatio         502
totalLim          0
totalRevBal       0
totalBcLim        0
totalIlLim        0
dtype: int64

In [14]:
# show the totalBcLim, bcOpen, and bcRatio columns
loan_data[['totalBcLim', 'bcOpen', 'bcRatio', 'totalRevBal']]

,totalBcLim,bcOpen,bcRatio,totalRevBal
0,1000.0,43.0,95.7,48054.0
1,31600.0,19756.0,37.5,11844.0
2,83000.0,7686.0,83.2,173260.0
3,11200.0,11061.0,1.2,20671.0
4,14200.0,5904.0,58.4,8475.0
...,...,...,...,...
49995,20800.0,15385.0,26.0,55174.0
49996,13200.0,1541.0,88.3,34757.0
49997,44500.0,38077.0,14.4,24032.0
49998,15300.0,5105.0,66.6,23206.0


In [23]:
# find missing values in bcOpen column and show 'totalBcLim', 'bcOpen', 'bcRatio', 'totalRevBal' columns
bcRatioNull = loan_data[loan_data['bcRatio'].isnull()][['totalBcLim', 'bcOpen', 'bcRatio', 'totalRevBal']]

In [27]:
# show the rows in bcRatioNull where totalBcLim is not 0.0
bcRatioNull[bcRatioNull['totalBcLim'] != 0.0]

,totalBcLim,bcOpen,bcRatio,totalRevBal
604,4100.0,234.0,NaN,25050.0
1227,1000.0,140.0,NaN,20951.0
6069,4600.0,2194.0,NaN,9119.0
6644,1600.0,57.0,NaN,52760.0
7151,300.0,108.0,NaN,46289.0
10636,2600.0,339.0,NaN,44442.0
11326,7500.0,1587.0,NaN,85864.0
11443,52300.0,7430.0,NaN,132033.0
11804,8000.0,3977.0,NaN,44786.0
13072,4800.0,974.0,NaN,16334.0


In [29]:
loan_data[loan_data['totalBcLim'] == 0.0][['totalBcLim', 'bcOpen', 'bcRatio', 'totalRevBal']]

,totalBcLim,bcOpen,bcRatio,totalRevBal
193,0.0,NaN,NaN,15340.0
260,0.0,NaN,NaN,25453.0
286,0.0,NaN,NaN,59267.0
288,0.0,NaN,NaN,66066.0
292,0.0,NaN,NaN,38646.0
...,...,...,...,...
49832,0.0,NaN,NaN,1648.0
49864,0.0,NaN,NaN,28455.0
49906,0.0,NaN,NaN,41575.0
49944,0.0,NaN,NaN,34482.0


In [15]:
# replacing the null values with 'unspecified'
#loan_data['employment'].fillna('unspecified', inplace=True)

* Next, we will drop the loanID column because it doesn't offer any useful information for the analysis or model

In [16]:
# dropping the Loan_ID column
loan_data.drop('loanID', axis=1, inplace=True)
loan_data.head()

,amount,term,rate,payment,grade,employment,length,home,income,verified,...,totalBal,totalRevLim,accOpen24,avgBal,bcOpen,bcRatio,totalLim,totalRevBal,totalBcLim,totalIlLim
0,8000.0,36 months,0.14,272.07,C,Warehouseman,3 years,RENT,49966.0,Source Verified,...,48054.0,8100.0,8.0,4369.0,43.0,95.7,60629.0,48054.0,1000.0,52529.0
1,11000.0,36 months,0.10,354.89,B,Vice President,10+ years,MORTGAGE,50000.0,Not Verified,...,265838.0,33400.0,4.0,22153.0,19756.0,37.5,297500.0,11844.0,31600.0,0.0
2,35000.0,36 months,0.15,1220.33,D,Owner/Attorney,10+ years,RENT,360000.0,Verified,...,850150.0,83000.0,2.0,121450.0,7686.0,83.2,907300.0,173260.0,83000.0,100000.0
3,20000.0,60 months,0.12,447.83,C,Analyst,2 years,MORTGAGE,60000.0,Source Verified,...,20671.0,14200.0,8.0,1723.0,11061.0,1.2,46964.0,20671.0,11200.0,32764.0
4,12000.0,60 months,0.12,266.88,B,medical technician,10+ years,MORTGAGE,64000.0,Source Verified,...,128034.0,14600.0,3.0,18291.0,5904.0,58.4,134600.0,8475.0,14200.0,0.0


In [17]:
loan_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49981 entries, 0 to 49999
Data columns (total 31 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   amount       49981 non-null  float64
 1   term         49981 non-null  object 
 2   rate         49981 non-null  float64
 3   payment      49981 non-null  float64
 4   grade        49981 non-null  object 
 5   employment   47200 non-null  object 
 6   length       47302 non-null  object 
 7   home         49981 non-null  object 
 8   income       49981 non-null  float64
 9   verified     49981 non-null  object 
 10  status       49981 non-null  object 
 11  reason       49981 non-null  object 
 12  state        49981 non-null  object 
 13  debtIncRat   49981 non-null  float64
 14  delinq2yr    49981 non-null  float64
 15  inq6mth      49981 non-null  float64
 16  openAcc      49981 non-null  float64
 17  pubRec       49981 non-null  float64
 18  revolRatio   49981 non-null  float64
 19  tota